In [14]:
from resources.filterLang import FilterLanguage as FL
from resources.tokTT import CommentTokenizer as CT
import fasttext
import numpy as np
from resources.basicIO import InputOutput as IO
import copy
from gensim.models import Word2Vec
from gensim.test.utils import common_texts

In [15]:
words = sorted(set(open('datasets/tokenized_corpus.txt').read().split()))

In [16]:
def NN(model, line, K):

    res = dict.fromkeys(words, [])
    res2 = dict.fromkeys(words, 0)

    for wrd in line:
        try:
            x = model.wv.most_similar(wrd, topn=K)
            for a, b in x:
                res[a] = res[a] + [b]
        except:
            pass

    for i in res:
        if(len(res[i]) == 0):
            res2[i] = 0 
        else:
            res2[i] = np.mean(res[i])

    #res2 = [(a,b) for b,a in res2]
    return res2


In [17]:
def oracleHelp(classdata):

    sums = sum(classdata)
    res = any(((ele/sums) >= 0.48 and (ele/sums) <= 0.52) for ele in classdata)
    return res


In [18]:
def cos_sim(a, b):
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)

In [19]:
def sim(x,y,sim_type='cosine_sim'):

    if(sim_type == 'cosine_sim'):
        return cos_sim(x,y)

In [20]:
# find similarity score matrix between A and B
# pass transpose of B
def sim_matrix(A, B, sim_type):
    m, p = A.shape
    p, n = B.shape
    C = np.zeros((m, n))
    for i in range(m):
        for j in range(n):
            C[i][j] = sim(A[i, :], B[:, j], sim_type)
    return C


In [21]:
# finds similarity score
def score(model, line, k=10):
    # words contains all the words in the corpus
    lst1 = NN(model, line, k)
    v1 = []
    l1 = [x[1] for x in lst1]
    l10 = [x[0] for x in lst1]
    for i in range(len(words)):
        try:
            v1.append(l10[l1.index(words[i])])
        except:
            v1.append(0)
    return v1


In [22]:
# finds classwise average and returns an array of predicted
# class labels. labels are classwise labels in ascending order
def label_avg(X, labels,count,exp_labels=None):

    m, n = X.shape
    if(m != len(labels)):
        return None

    avg_sums = []
    no_labels = len(list(set(labels)))

    for p in range(n):

        column = X[:, p]

        avgs = np.zeros(no_labels)

        for lbl in range(no_labels):
            indices = [ix for ix, label in enumerate(labels) if label == lbl]
            avgs[lbl] = np.mean([column[x] for x in indices])

        if(exp_labels != None and oracleHelp(avgs)):
            avg_sums.append(exp_labels[p])
            count[0]+=1
        else:
            max_avg_pos = avgs.argmax()
            avg_sums.append(max_avg_pos)

    return avg_sums

In [23]:
def get_NN(model,lines_TK,k):
    scores = []
    for line in lines_TK:
        scores.append(score(model, line, k))

    return scores  

### load model

In [24]:
model_N_2 = Word2Vec.load("models/word2vec.model")


### load comments, seedset, labels of both

In [25]:
# loading 60 comments
expansion_text = IO.load_csv_col('datasets/random_sample.csv', 'comment')
expansion_text = expansion_text[0:500]
expansion_text_labels = IO.load_csv_col('datasets/random_sample.csv', 'label')
expansion_text_labels = list(map(int, expansion_text_labels[0:500]))
seed_set_text = IO.load_text('datasets/seed_set.txt')
seed_set_labels = IO.load_text('datasets/seed_set_labels.txt')
seed_set_labels = list(map(int, seed_set_labels))
seed_set_TK = CT.cleaned('datasets/seed_set.txt')
expansion_TK = [CT.tokenize(x) for x in expansion_text]

In [26]:
# select comments from expansion_set in batches
batch_size = 20
k = 30

M = np.arange(0,500,batch_size)

seed_TK = copy.deepcopy(seed_set_TK)
seed_labels = copy.deepcopy(seed_set_labels)
seed_text = copy.deepcopy(seed_set_text)

count2 = [0]
for i in range(1, len(M)):
    
    exp_TK = expansion_TK[M[i-1]:M[i]]
    exp_labels = expansion_text_labels[M[i-1]:M[i]]

    # find NN
    seed_NN = get_NN(model_N_2,seed_TK,k)
    exp_NN = get_NN(model_N_2,exp_TK,k)

    A = np.array(seed_NN)
    B = np.array(exp_NN).T
    C = sim_matrix(A,B,'cosine_sim')
    #Y = label_avg(C, seed_labels, count2)
    Y = label_avg(C, seed_labels, count2, exp_labels)
    seed_labels += Y
    seed_TK += exp_TK
    seed_text += expansion_text[M[i-1]:M[i]]


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
#seeds = [x for _,x in sorted(zip(seed_labels,seed_text))]
#Y = [y for y,_ in sorted(zip(seed_labels,seed_text))]

In [ ]:
IO.save_text('datasets_post/batch_N_2.txt', seed_text)
IO.save_text('datasets_post/batch_labels_N_2.txt',map(str, seed_labels))


In [ ]:
print(count2)

[17]
